In [1]:
from file_parser import parse_traj, read_gbnsr6_output, store_frame_inpcrd, store_single_frame_mdcrd
import subprocess

In [2]:
mdcrd = open('prod1.mdcrd', 'r')
lines = mdcrd.readlines()
g = parse_traj(lines, 3862, 42193, skip = 1, return_full=True)

In [3]:
def get_numbers(line, limit = -1):
    new_digit = ''
    numbers = []
    is_neg = False
    for i in range(len(line)):
        c = line[i]
        if i < len(line) - 1 and c == '-' and line[i + 1].isdigit():
            is_neg = True
        if c.isdigit() or (c == '.' and new_digit != ''):
            new_digit += c
        if new_digit != '' and c != '.' and (i == len(line) - 1 or not c.isdigit()):
            new_num = float(new_digit)
            new_num = -new_num if is_neg else new_num
            numbers.append(new_num)
            if len(numbers) == limit:
                return numbers
            new_digit = ''
    return numbers

In [4]:
full_record = []
for fr in g:
    store_single_frame_mdcrd(fr[0], per_line=10, fp='single.mdcrd')
    subprocess.run(['./run_mmpbsa.sh'])
    f = open('FINAL_RESULTS_MMPBSA.dat').readlines()
    gb = ''.join(f).split('GENERALIZED BORN:')[1].split('POISSON BOLTZMANN:')[0]
    mmpbsa_dic = {}
    for title in ['Complex', 'Receptor', 'Ligand']:
        for param in ['EEL', 'EGB', 'ESURF']:
            mmpbsa_dic[title + '_' + param] = get_numbers(gb.split(title)[1].split('TOTAL')[0].split(param)[1].split('\n')[0])[0]
    full_record.append(mmpbsa_dic)

Error: in read of Coords frame 1 of trajectory single.mdcrd.
Error: Could not set up 'single.mdcrd' for reading.
Error: Could not set up input trajectory 'single.mdcrd'.
Error: Error(s) occurred during execution.
  File "/home/ali/Amber/amber22/bin/MMPBSA.py", line 99, in <module>
    app.file_setup()
  File "/home/ali/Amber/amber22/lib/python3.9/site-packages/MMPBSA_mods/main.py", line 155, in file_setup
    self.numframes, self.numframes_nmode = make_trajectories(INPUT, FILES,
  File "/home/ali/Amber/amber22/lib/python3.9/site-packages/MMPBSA_mods/make_trajs.py", line 61, in make_trajectories
    traj = Trajectory(FILES.solvated_prmtop, FILES.mdcrd, cpptraj)
  File "/home/ali/Amber/amber22/lib/python3.9/site-packages/MMPBSA_mods/make_trajs.py", line 469, in __init__
    self.Query()
  File "/home/ali/Amber/amber22/lib/python3.9/site-packages/MMPBSA_mods/make_trajs.py", line 604, in Query
    raise TrajError('%s failed when querying %s' % (self.exe, traj))
TrajError: /home/ali/Amber/a

IndexError: list index out of range